# 🚛 Truck and License Plate Detection Training

This notebook provides a complete pipeline for training a YOLOv8 model to detect trucks and license plates with high accuracy and speed.

## 🚀 Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

# Install dependencies
!pip install -q ultralytics
!pip install -q roboflow
!pip install -q easyocr
!pip install -q onnx onnxruntime

## 📥 Clone Repository

In [ ]:
# Clone your repository
!git clone https://github.com/Road-Maintenance-Fund/RMF-TRUCKS-MONITOR-DEMO.git
%cd RMF-TRUCKS-MONITOR-DEMO
!ls -la

## 🏋️ Train Model

In [ ]:
from ultralytics import YOLO
import torch

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load model
model = YOLO('yolov8n.pt')

# Train
results = model.train(
    data='dataset/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    device=0 if device == 'cuda' else 'cpu',
    workers=4,
    optimizer='auto',
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    name='truck_plate_detection'
)

## 💾 Save to Google Drive

In [ ]:
from google.colab import drive
import os
from datetime import datetime

# Mount Google Drive
drive.mount('/content/drive')

# Create save directory
save_dir = '/content/drive/MyDrive/plates_model'
os.makedirs(save_dir, exist_ok=True)

# Export to ONNX
model.export(format='onnx', dynamic=True)

# Copy files
!cp -r runs/ $save_dir/
!cp *.onnx $save_dir/

print(f'\nModel saved to: {save_dir}')
!ls -la $save_dir

## 🚀 Test Model

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def test_image(model, image_path):
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image: {image_path}")
        return
    
    # Run inference
    results = model(img)
    
    # Show results
    for r in results:
        im_array = r.plot()
        # Convert BGR to RGB for display
        im_array = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB)
        
        # Display
        cv2_imshow(im_array)
        
        # Print detections
        for box in r.boxes:
            print(f"Detected {r.names[box.cls[0].item()]} with confidence {box.conf[0].item():.2f}")

# Test on a sample image
test_image_path = 'path/to/your/test/image.jpg'  # Update this path
test_image(model, test_image_path)